<center>
<h1>Факультет "Прикладная математика" МАИ</h1>
<h2>Курс "Основы Python для анализа данных"</h2>
<h2>Артамонов Игорь Михайлович</h2>
</center>

<center><h2>Занятие № 15. Временные ряды и их анализ.</h2></center>

## Общение / вопросы по курсу

Платформа для групповой работы Atlassian Confluence факультета "Прикладная математика"

https://mai.moscow/display/PYTML

* <b>Занятие № 15. Временные ряды и их анализ</b>

## virtualenv + Jupyter notebook

```
<Ctrl> + <Alt> + T - новое окно терминала
```

```
$ conda -V

$ conda update conda

$ conda search "^python$"

$ conda create -n yourenvname python=x.x anaconda

$ source activate yourenvname

$ jupyter notebook

$ conda install -n yourenvname [package]
```

### <font color="green">ВОПРОС</a>
Какие задачи машинного обучения для __временных рядов__ могут быть:
* регрессией?
* классификацией?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sc
from scipy.stats import randint
from numpy.random import randn
import seaborn as sns
from scipy import stats

import matplotlib as mpl
from matplotlib.patches import Ellipse
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm  
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.model_selection import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

import sys
import itertools
import os
import os.path

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 

## for Deep-learing:
import keras
from keras.preprocessing import sequence
from keras.layers import Dense, Embedding, LSTM, ConvLSTM2D, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D


### Временной ряд
* последовательно измеренные через некоторые (зачастую равные) промежутки времени данные

__Что из них можно извлечь__
* что происходило (закономерности в прошлом)
* влияющие факторы, особенно - связанные со временем
* что будет происходить (предсказание будущего)

__Обычно делят на:__
* анализ в частотной области
    - спектральный анализ / вейвлет-анализ
    - хорошо подходит для предварительных исследований
* анализ во временной области
    - кросс-корреляционный анализ
    - автокорреляционный анализ

__Что с ними можно сделать__<br><br>
__Общее исследование__
* Визуальное изучение графических представлений временных рядов
* Автокорреляционный анализ для изучения зависимостей
* Спектральный анализ для изучения циклического поведения, не связанного с сезонностью<br><br>

__Описание__
* Разделение компонент: тренд, сезонность, медленно и быстро меняющиеся компоненты, циклическая нерегулярность
* Простейшие свойства частных распределений<br><br>

__Прогнозирование и предсказание__
* Полноценные статистические модели при стохастическом моделировании для создания альтернативных версий временных рядов, показывающих, что могло бы случиться на произвольных отрезках времени в будущем (предсказание)
* Упрощённые или поноценные статистические модели для описания вероятные значения временного ряда в ближайшем будущем при известных последних значениях (прогноз)

Что нам будет полезно:
* statsmodels - методы статистического моделирования
* одномерные сверточные модели
* рекуррентные сети

### Метод скользящих средних

$$
\hat{y_t} = \frac{1}{k} \sum_{n=0}^{k-1} y_{t\_n}
$$

### Экспоненциальное сглаживание

Одинарное: 
* учитываем исторические данные с экспоненциальным уменьшением их веса
$$
\hat{y_t} = \alpha \cdot y_t + (1- \alpha) \cdot y_{t-1}
$$<br>
Двойное:<br><br>
* добавляем уровень (level, intercept) и тренд (trend, slope)
$$
l_x = \alpha y_x + (1 - \alpha) (l_{x-1} + b_{x-1})
$$
<br>
$$
b_x = \beta (l_x - l_{x-1}) + (1 - \beta) b_{x-1}
$$
<br>
$$
\hat{y_t} = l_x + b_x
$$<br><br>
Модель Холта-Винтерса:<br><br>
* добавляем сезонность (повторяющиеся колебания вокруг уровня и тренда), которая характеризуется длиной сезона — периодом<br><br>
$$
l_x = \alpha (y_x +  s_{x-L}) + (1 - \alpha) (l_{x-1} + b_{x-1})
$$
<br>
$$
b_x = \beta (l_x - l_{x-1}) + (1 - \beta) b_{x-1}
$$
<br>
$$
s_x = \gamma (y_x - l_x) + (1 - \gamma) s_{x-L}
$$
<br>
$$
\hat{y_t} = l_x + b_x
$$

### ARIMA

http://www.seanabu.com/2016/03/22/time-series-seasonal-ARIMA-model-in-python/

<b>A</b>uto<b>R</b>regression. A model that uses the dependent relationship between an observation and some number of lagged observations.<br>
<b>I</b>ntegrated. The use of differencing of raw observations (e.g. subtracting an observation from an observation at the previous time step) in order to make the time series stationary.<br>
<b>M</b>oving <b>A</b>verage. A model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations.

__Стационарность__ - свойство процесса не менять своих статистических характеристик с течением времени, а именно постоянство матожидания, постоянство дисперсии (она же гомоскедастичность) и независимость ковариационной функции от времени (должна зависеть только от расстояния между наблюдениями)

Матожидание растет:
<img src="images15/Mean_nonstationary.png">

Варьируется дисперсия:
<img src="images15/Var_nonstationary.png">

Непостоянная ковариация:
<img src="images15/Cov_nonstationary.png">

In [ ]:
df = pd.read_csv('data15/portland-oregon-average-monthly-.csv', index_col=0)
df.index.name=None
df.reset_index(inplace=True)
df.drop(df.index[114], inplace=True)

In [ ]:
start = datetime.datetime.strptime("1973-01-01", "%Y-%m-%d")
date_list = [start + relativedelta(months=x) for x in range(0,114)]
df['index'] =date_list
df.set_index(['index'], inplace=True)
df.index.name=None

In [ ]:
df.columns= ['riders']
df['riders'] = df.riders.apply(lambda x: int(x)*100)

In [ ]:
df.riders.plot(figsize=(12,8), title= 'Monthly Ridership', fontsize=14)
plt.savefig('month_ridership.png', bbox_inches='tight')

In [ ]:
decomposition = seasonal_decompose(df.riders, freq=12)  
fig = plt.figure()  
fig = decomposition.plot()  
fig.set_size_inches(15, 8)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(df.riders)

In [ ]:
df.columns

In [ ]:
df['riders_log']= df.riders.apply(lambda x: np.log(x))  
test_stationarity(df.riders_log)

In [ ]:
df['first_difference'] = df.riders - df.riders.shift(1)  
test_stationarity(df.first_difference.dropna(inplace=False))

In [ ]:
df['log_first_difference'] = df.riders_log - df.riders_log.shift(1)  
test_stationarity(df.log_first_difference.dropna(inplace=False))

In [ ]:
df['seasonal_difference'] = df.riders - df.riders.shift(12)  
test_stationarity(df.seasonal_difference.dropna(inplace=False))

In [ ]:
df['log_seasonal_difference'] = df.riders_log - df.riders_log.shift(12)  
test_stationarity(df.log_seasonal_difference.dropna(inplace=False))

In [ ]:
df['seasonal_first_difference'] = df.first_difference - df.first_difference.shift(12)  
test_stationarity(df.seasonal_first_difference.dropna(inplace=False))

In [ ]:
df['log_seasonal_first_difference'] = df.log_first_difference - df.log_first_difference.shift(12)  
test_stationarity(df.log_seasonal_first_difference.dropna(inplace=False))

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df.seasonal_first_difference.iloc[13:], lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df.seasonal_first_difference.iloc[13:], lags=40, ax=ax2)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df.riders, trend='n', order=(0,1,0), seasonal_order=(0,1,1,12))
results = mod.fit()
print (results.summary())

In [ ]:
mod = sm.tsa.statespace.SARIMAX(df.riders, trend='n', order=(0,1,0), seasonal_order=(1,1,1,12))
results = mod.fit()
print (results.summary())

In [ ]:
df['forecast'] = results.predict(start = 102, end= 114, dynamic= True)  
df[['riders', 'forecast']].plot(figsize=(12, 8)) 
plt.savefig('ts_df_predict.png', bbox_inches='tight')

In [ ]:
npredict =df.riders['1982'].shape[0]
fig, ax = plt.subplots(figsize=(12,6))
npre = 12
ax.set(title='Ridership', xlabel='Date', ylabel='Riders')
ax.plot(df.index[-npredict-npre+1:], df.iloc[-npredict-npre+1:]['riders'], 'o', label='Observed')
ax.plot(df.index[-npredict-npre+1:], df.iloc[-npredict-npre+1:]['forecast'], 'g', label='Dynamic forecast')
legend = ax.legend(loc='lower right')
legend.get_frame().set_facecolor('w')
plt.savefig('ts_predict_compare.png', bbox_inches='tight')

Сложности с ARIMA-моделями:
* требуют много времени на подготовку данных
* сложно настраивать
* требуют частого переобучения на новых данных

## Рекуррентные нейронные сети (RNN) - для временных рядов

* В традиционной NN предполагается, что все входы и выходы независимы друг от друга
* Есть задачи, в которых есть зависимость между предыдущими и последующими данными
* Основная идея - получить возможность работать с последовательной информацией.

In [ ]:
!ls ..

<img src="../Lesson_14/images14/rnn.jpg">
A recurrent neural network and the unfolding in time of the computation involved in its forward computation. Source: Nature

Обучение RNN производится с помощью алгоритма _обратного распространения через время_ (Backpropagation Through Time - __BPTT__))

<img src="../Lesson_14/images14/Backpropagation-Through-Time-BPTT-in-standard-BRNNs.png">

<img src="../Lesson_14/images14/greff_lstm_diagram.png">

Используем:
* dataset: https://www.kaggle.com/uciml/electric-power-consumption-data-set
* kernel: https://www.kaggle.com/amirrezaeian/time-series-data-analysis-using-lstm-tutorial

Data Set Information:

This archive contains 2075259 measurements gathered in a house located in Sceaux (7km of Paris, France) between December 2006 and November 2010 (47 months). 
Notes: 
<ol>
<li>(global_active_power*1000/60 - sub_metering_1 - sub_metering_2 - sub_metering_3) represents the active energy consumed every minute (in watt hour) in the household by electrical equipment not measured in sub-meterings 1, 2 and 3. 
<li>The dataset contains some missing values in the measurements (nearly 1,25% of the rows). All calendar timestamps are present in the dataset but for some timestamps, the measurement values are missing: a missing value is represented by the absence of value between two consecutive semi-colon attribute separators. For instance, the dataset shows missing values on April 28, 2007.
</ol>

Attribute Information:

<ol>
<li>date: Date in format dd/mm/yyyy 
<li>time: time in format hh:mm:ss 
<li>global_active_power: household global minute-averaged active power (in kilowatt) 
<li>global_reactive_power: household global minute-averaged reactive power (in kilowatt) 
<li>voltage: minute-averaged voltage (in volt) 
<li>global_intensity: household global minute-averaged current intensity (in ampere) 
<li>sub_metering_1: energy sub-metering No. 1 (in watt-hour of active energy). It corresponds to the kitchen, containing mainly a dishwasher, an oven and a microwave (hot plates are not electric but gas powered). 
<li>sub_metering_2: energy sub-metering No. 2 (in watt-hour of active energy). It corresponds to the laundry room, containing a washing-machine, a tumble-drier, a refrigerator and a light. 
<li>sub_metering_3: energy sub-metering No. 3 (in watt-hour of active energy). It corresponds to an electric water-heater and an air-conditioner.
</ol>


In [ ]:
df = pd.read_csv('data15/household_power_consumption.txt', sep=';',
                 parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True,
                 low_memory=False, na_values=['nan','?'], index_col='dt')

In [ ]:
df.head()

In [ ]:
# Заменяем nan на средние по колонкам
for j in range(0,7):        
        df.iloc[:,j]=df.iloc[:,j].fillna(df.iloc[:,j].mean())

In [ ]:
# Проверяем, что не осталось нулевых значений
df.isnull().sum()

In [ ]:
df.Global_active_power.resample('D').sum().plot(title='Global_active_power resampled over day for sum') 
plt.tight_layout()
plt.show()   

df.Global_active_power.resample('D').mean().plot(title='Global_active_power resampled over day for mean', color='red') 
plt.tight_layout()
plt.show()

### <font color="green">ВОПРОС</font>
Видите ли Вы здесь аномалии?

In [ ]:
r = df.Global_intensity.resample('D').agg(['mean', 'std'])
r.plot(subplots = True, title='Global_intensity resampled over day')
plt.show()

In [ ]:
df['Global_active_power'].resample('M').mean().plot(kind='bar')
plt.xticks(rotation=60)
plt.ylabel('Global_active_power')
plt.title('Global_active_power per month (averaged over month)')
plt.show()

In [ ]:
df['Global_active_power'].resample('Q').mean().plot(kind='bar')
plt.xticks(rotation=60)
plt.ylabel('Global_active_power')
plt.title('Global_active_power per quarter (averaged over quarter)')
plt.show()

In [ ]:
df['Voltage'].resample('M').mean().plot(kind='bar', color='red')
plt.xticks(rotation=60)
plt.ylabel('Voltage')
plt.title('Voltage per quarter (summed over quarter)')
plt.show()

In [ ]:
df['Sub_metering_1'].resample('M').mean().plot(kind='bar', color='brown')
plt.xticks(rotation=60)
plt.ylabel('Sub_metering_1')
plt.title('Sub_metering_1 per quarter (summed over quarter)')
plt.show()

In [ ]:
# Below I compare the mean of different featuresre sampled over day. 
# specify columns to plot
cols = [0, 1, 2, 3, 5, 6]
i = 1
groups=cols
values = df.resample('D').mean().values
# plot each column
plt.figure(figsize=(15, 10))
for group in groups:
    plt.subplot(len(cols), 1, i)
    plt.plot(values[:, group])
    plt.title(df.columns[group], y=0.75, loc='right')
    i += 1
plt.show()

In [ ]:
df.Global_reactive_power.resample('W').mean().plot(color='y', legend=True)
df.Global_active_power.resample('W').mean().plot(color='r', legend=True)
df.Sub_metering_1.resample('W').mean().plot(color='b', legend=True)
df.Global_intensity.resample('W').mean().plot(color='g', legend=True)
plt.show()

In [ ]:
df.Global_active_power.resample('M').mean().plot(kind='hist', bins=30, color='r', legend=True )
df.Global_reactive_power.resample('M').mean().plot(kind='hist', bins=30, color='b', legend=True)

df.Global_intensity.resample('M').mean().plot(kind='hist', bins=30, color='g', legend=True)
df.Sub_metering_1.resample('M').mean().plot(kind='hist', bins=30, color='y', legend=True)
plt.show()

In [ ]:
df.Voltage.resample('M').sum().plot(kind='hist',color='g', legend=True)

In [ ]:
data_returns = df.pct_change()
sns.jointplot(x='Global_intensity', y='Global_active_power', data=data_returns)

plt.show()

In [ ]:
sns.jointplot(x='Voltage', y='Global_active_power', data=data_returns)  
plt.show()

In [ ]:
plt.matshow(df.corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
plt.title('without resampling', size=15)
plt.colorbar()
plt.show()

In [ ]:
plt.matshow(df.resample('M').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
plt.title('resampled over month', size=15)
plt.colorbar()
plt.margins(0.02)
plt.matshow(df.resample('A').mean().corr(method='spearman'),vmax=1,vmin=-1,cmap='PRGn')
plt.title('resampled over year', size=15)
plt.colorbar()
plt.show()

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    dff = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(dff.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(dff.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
## resampling of data over hour
df_resample = df.resample('h').mean() 
df_resample.shape

In [ ]:
## If you would like to train based on the resampled data (over hour), then used below
values = df_resample.values 

## full data without resampling
#values = df.values

# integer encode direction
# ensure all data is float
#values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
reframed.drop(reframed.columns[[8,9,10,11,12,13]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
# split into train and test sets
values = reframed.values

n_train_time = 365*24
train = values[:n_train_time, :]
test = values[n_train_time:, :]
##test = values[n_train_time:n_test_time, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# We reshaped the input into the 3D format as expected by LSTMs, namely [samples, timesteps, features].

__Архитектура модели__
<ol>
<li>LSTM с 100 нейронами в первом видимом слое
<li>dropout 20%
<li>1 нейрон в выходном слое для предсказания  Global_active_power.
<li>Входные данные - 1 временной шаг с 7 переменными (features).
<li>Mean Absolute Error (MAE) loss function
<li>Adam version of stochastic gradient descent.
<li>The model will be fit for 20 training epochs with a batch size of 70.
</ol>

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
#    model.add(LSTM(70))
#    model.add(Dropout(0.3))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, epochs=20, batch_size=70, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], 7))
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, -6:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, -6:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
## time steps, every step is one hour (you can easily convert the time step to the actual time index)
## for a demonstration purpose, I only compare the predictions in 200 hours. 

aa=[x for x in range(200)]
plt.plot(aa, inv_y[:200], marker='.', label="actual")
plt.plot(aa, inv_yhat[:200], 'r', label="prediction")
plt.ylabel('Global_active_power', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show()

https://github.com/Prakashvanapalli/TensorFlow/blob/master/Blogposts/Backpropogation_with_Images.ipynb

## Экзаменационные вопросы:

* Временные ряды